In [49]:
from curses.ascii import ctrl
from ucl.common import byte_print, decode_version, decode_sn, getVoltage, pretty_print_obj, lib_version
from ucl.lowState import lowState
from ucl.lowCmd import lowCmd
from ucl.unitreeConnection import unitreeConnection, LOW_WIFI_DEFAULTS, LOW_WIRED_DEFAULTS
from ucl.enums import GaitType, SpeedLevel, MotorModeLow
from ucl.complex import motorCmd, motorCmdArray
import time
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
import glob

## Linear interpolation between two joint positions
# Input : initPos(rate=0) ~~~~~~ targetPos(rate=1)
# Output : interpolated_position =(p)
def jointLinearInterpolation(initPos, targetPos, rate):
    rate = np.fmin(np.fmax(rate, 0.0), 1.0)
    p = initPos*(1-rate) + targetPos*rate
    return p

## Utility functions for file management
def ensure_logs_directory():
    """Create logs directory if it doesn't exist"""
    if not os.path.exists('logs'):
        os.makedirs('logs')
        print("Created 'logs' directory")

def get_timestamp_filename(base_name, extension):
    """Generate filename with timestamp prefix"""
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return f"{timestamp}_{base_name}.{extension}"

def get_latest_csv_file():
    """Get the most recent CSV file from logs directory"""
    ensure_logs_directory()
    csv_pattern = os.path.join('logs', '*_joint_tracking_data.csv')
    csv_files = glob.glob(csv_pattern)
    
    if not csv_files:
        print("No CSV files found in logs directory")
        return None
    
    # Sort by filename (timestamp) in descending order
    csv_files.sort(reverse=True)
    latest_file = csv_files[0]
    print(f"Latest CSV file: {latest_file}")
    return latest_file

## Modularized plotting function for joint tracking analysis
def plot_joint_tracking_from_csv(csv_file_path, output_file=None):
    """
    CSV 파일에서 joint tracking 데이터를 읽어서 플롯 생성
    
    Args:
        csv_file_path (str): CSV 파일 경로
        output_file (str, optional): 출력 이미지 파일 경로. None이면 화면에 표시만 함.
    """
    if not os.path.exists(csv_file_path):
        print(f"CSV 파일을 찾을 수 없습니다: {csv_file_path}")
        return
    
    # CSV 데이터 읽기
    df = pd.read_csv(csv_file_path)
    
    # Joint 이름 추출 (target 컬럼에서)
    target_cols = [col for col in df.columns if col.endswith('_target')]
    joint_names = [col.replace('_target', '') for col in target_cols]
    jointnum = len(joint_names)
    
    if jointnum == 0:
        print("CSV 파일에서 joint 데이터를 찾을 수 없습니다.")
        return
    
    # 시간 데이터 추출 (실제 시간 사용)
    if 'real_time' in df.columns:
        time_array = df['real_time'].values
        time_label = 'Real Time (s)'
    elif 'timestamp' in df.columns:
        time_array = df['timestamp'].values
        time_label = 'Time (s)'
    else:
        print("시간 데이터를 찾을 수 없습니다.")
        return
    
    # Create 3x4 subplot grid with doubled width
    fig, axes = plt.subplots(3, 4, figsize=(40, 15))  # 2배 폭으로 증가
    fig.suptitle('Joint Tracking Performance Analysis', fontsize=20)
    
    # Flatten axes for easier indexing
    axes_flat = axes.flatten()
    
    # Plot each joint
    for i in range(jointnum):
        ax = axes_flat[i]
        
        # Get target and actual data
        target_col = f'{joint_names[i]}_target'
        actual_col = f'{joint_names[i]}_actual'
        
        if target_col in df.columns and actual_col in df.columns:
            target_data = df[target_col].values
            actual_data = df[actual_col].values
            
            # Plot target and actual positions
            ax.plot(time_array, target_data, 'b-', label='Target', linewidth=2)
            ax.plot(time_array, actual_data, 'r-', label='Actual', linewidth=1.5)
            
            # Calculate tracking error
            error = actual_data - target_data
            ax2 = ax.twinx()
            ax2.plot(time_array, error, 'g--', label='Error', alpha=0.7)
            ax2.set_ylabel('Error (deg)', color='g')
            ax2.tick_params(axis='y', labelcolor='g')
            
            # Set labels and title
            ax.set_title(f'{joint_names[i]} Tracking Performance', fontsize=14)
            ax.set_xlabel(time_label)
            ax.set_ylabel('Position (deg)')
            ax.grid(True, alpha=0.3)
            ax.legend(loc='upper right')
            
            # Calculate and display statistics
            rmse = np.sqrt(np.mean(error**2))
            max_error = np.max(np.abs(error))
            ax.text(0.02, 0.98, f'RMSE: {rmse:.2f}°\nMax Error: {max_error:.2f}°', 
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        else:
            ax.text(0.5, 0.5, f'No data for {joint_names[i]}', 
                   transform=ax.transAxes, ha='center', va='center')
    
    # Hide unused subplots
    for i in range(jointnum, 12):
        axes_flat[i].set_visible(False)
    
    plt.tight_layout()
    
    if output_file:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"플롯이 '{output_file}'로 저장되었습니다.")
    
    plt.show()
    
    # Print overall statistics
    print("\n=== 전체 추종 성능 통계 ===")
    for i in range(jointnum):
        target_col = f'{joint_names[i]}_target'
        actual_col = f'{joint_names[i]}_actual'
        if target_col in df.columns and actual_col in df.columns:
            error = df[actual_col].values - df[target_col].values
            rmse = np.sqrt(np.mean(error**2))
            max_error = np.max(np.abs(error))
            print(f"{joint_names[i]}: RMSE={rmse:.2f}°, Max Error={max_error:.2f}°")


# You can use one of the 3 Presets WIFI_DEFAULTS, LOW_CMD_DEFAULTS or HIGH_CMD_DEFAULTS.
# IF NONE OF THEM ARE WORKING YOU CAN DEFINE A CUSTOM ONE LIKE THIS:
#
# MY_CONNECTION_SETTINGS = (listenPort, addr_wifi, sendPort_high, local_ip_wifi)
# conn = unitreeConnection(MY_CONNECTION_SETTINGS)
d = {'FR_0':0, 'FR_1':1, 'FR_2':2,
     'FL_0':3, 'FL_1':4, 'FL_2':5,
     'RR_0':6, 'RR_1':7, 'RR_2':8,
     'RL_0':9, 'RL_1':10, 'RL_2':11 }


##   Define constants     ##
PosStopF  = math.pow(10,9)
VelStopF  = 16000.0
LOWLEVEL  = 0xff
dt = 0.002 # 2ms control loop, While 문 도는 Hz를 의미함, 모터 제어 Hz 다름 = 50Hz(ctrl_dt)
## == Define constants == ##


## Initialize connection ##
print(f'Running lib version: {lib_version()}')
conn = unitreeConnection(LOW_WIFI_DEFAULTS)
conn.startRecv()
## == Initialize connection == ##


## instantiate lowlevel command and state objects ##
lcmd = lowCmd()
# lcmd.encrypt = True
lstate = lowState()
mCmdArr = motorCmdArray()


## get initial state & print Log ##
# Send empty command to tell the dog the receive port and initialize the connection
cmd_bytes = lcmd.buildCmd(debug=False)
conn.send(cmd_bytes)

# define a function to print log data
def printLog(conn):
    data = conn.getData()
    for paket in data:
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
        lstate.parseData(paket)
        print(f'SN [{byte_print(lstate.SN)}]:\t{decode_sn(lstate.SN)}')
        print(f'Ver [{byte_print(lstate.version)}]:\t{decode_version(lstate.version)}')
        print(f'SOC:\t\t\t{lstate.bms.SOC} %')
        print(f'Overall Voltage:\t{getVoltage(lstate.bms.cell_vol)} mv') #something is still wrong here ?!
        print(f'Current:\t\t{lstate.bms.current} mA')
        print(f'Cycles:\t\t\t{lstate.bms.cycle}')
        print(f'Temps BQ:\t\t{lstate.bms.BQ_NTC[0]} °C, {lstate.bms.BQ_NTC[1]}°C')
        print(f'Temps MCU:\t\t{lstate.bms.MCU_NTC[0]} °C, {lstate.bms.MCU_NTC[1]}°C')
        print(f'FootForce:\t\t{lstate.footForce}')
        print(f'FootForceEst:\t\t{lstate.footForceEst}')
        print(f'IMU Temp:\t\t{lstate.imu.temperature}')
        print(f'MotorState FR_0 MODE:\t\t{lstate.motorState[d["FR_0"]].mode}')
        print('+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=')
    return data

# return the initial data 
data = printLog(conn)


## Main control loop ##
motiontime = 0 # <-- control timing counter
print('control Freq : {} Hz'.format(1/dt))


## Robot Control Parameters
# control HZ : 50Hz
ctrldt = 1/50 # Ctrl loop HZ : 50Hz

# joints for control 
joint = ['FR_0', 'FR_1', 'FR_2','FL_0', 'FL_1', 'FL_2','RR_0', 'RR_1', 'RR_2','RL_0', 'RL_1', 'RL_2']
jointnum = len(joint)
print("num of controlled joints :", jointnum)

## Joint Configuration for Sine Wave Motion
# 각 joint마다 다른 주파수와 위상 지연 설정
joint_config = {
    'FR_0': {'freq_hz': 0.0, 'phase_delay': 0.0},    # Hip joint - 정지
    'FR_1': {'freq_hz': 1, 'phase_delay': 0.0},     # Thigh joint - 기본 주파수
    'FR_2': {'freq_hz': 1, 'phase_delay': 0.0},     # Calf joint - 위상 지연

    'FL_0': {'freq_hz': 0.0, 'phase_delay': 0.0},     # Hip joint - 정지
    'FL_1': {'freq_hz': 1, 'phase_delay': 0.25},     # Thigh joint - 기본 주파수
    'FL_2': {'freq_hz': 1, 'phase_delay': 0.25},     # Calf joint - 위상 지연

    'RR_0': {'freq_hz': 0.0, 'phase_delay': 0.0},     # Hip joint - 정지
    'RR_1': {'freq_hz': 1, 'phase_delay': 0.5},     # Thigh joint - 기본 주파수
    'RR_2': {'freq_hz': 1, 'phase_delay': 0.5},     # Calf joint - 위상 지연

    'RL_0': {'freq_hz': 0.0, 'phase_delay': 0.0},     # Hip joint - 정지
    'RL_1': {'freq_hz': 1, 'phase_delay': .75},     # Thigh joint - 기본 주파수
    'RL_2': {'freq_hz': 1, 'phase_delay': .75},     # Calf joint - 위상 지연
}

# Config를 리스트로 변환 (joint 순서에 맞게)
freq_hz_list = [joint_config[joint[i]]['freq_hz'] for i in range(jointnum)]
phase_delay_list = [joint_config[joint[i]]['phase_delay'] for i in range(jointnum)]

print("Joint Frequency Config (Hz):", freq_hz_list)
print("Joint Phase Delay Config:", phase_delay_list)

# p / d gains
# [FR_0, FR_1, FR_2]
kp = [15] * jointnum # [8,8,...,8]
kd = [4] * jointnum # [1,1,...,1]
## This one Should be Modified when changed to 4legs ##
tau = [-0.65,2,2,-0.65,2,2,-0.65,2,2,-0.65,2,2] # [FR_0, FR_1, FR_2, FL_0, FL_1, FL_2, RR_0, RR_1, RR_2, RL_0, RL_1, RL_2]

## Set Min/Max value of joints
# sine wave amplitude, median value comes from here
motionscale = [0.6,0.8]

minmaxjoint = [ [0  ,0   ], #XX_0_min,XX_0_max
                [-10 * motionscale[0],80 * motionscale[0]  ], #XX_1_min,XX_1_max
                [-110 * motionscale[1],-60 * motionscale[1]]] #XX_2_min,XX_2_max
diffjoint = [0] * jointnum
medjoint = [0] * jointnum

for i in range(jointnum):
    # Use modulo to cycle through minmaxjoint indices (0,1,2) for 6 joints
    joint_idx = i % 3  # 0,1,2,0,1,2 for 6 joints
    diffjoint[i] = minmaxjoint[joint_idx][1] - minmaxjoint[joint_idx][0]
    medjoint[i] = (minmaxjoint[joint_idx][1] + minmaxjoint[joint_idx][0]) / 2
deg_sineamplitude = [x/2 for x in diffjoint]
deg_sinemedian = medjoint
print("Sine Amplitude (deg) :", deg_sineamplitude)
print("Sine Median (deg)    :", deg_sinemedian)

##   Inintialize variables container     ##
qInit = [0] * jointnum
qDes = [0] * jointnum

# Data logging for tracking performance
target_positions = []  # Store target positions
actual_positions = []  # Store actual motor positions
timestamps = []        # Store dt-based timestamps
real_times = []        # Store real time.time() timestamps
start_time = None      # Store start time for real time calculation
## == Inintialize variables container == ##

## functionGenerator for sine wave motion
def functionGenerator(t,freq_hz, deg_amplitude, deg_median,range_phase_delay=0.0):
    # 2pi = 360deg = 1 loop
    # Thus, for 5Hz -> 5 loop in 1 sec -> (2pi*5) rad in 1 sec
    freq_rad = (2* math.pi)/freq_hz #sine frequency in rad/s

    # range_phase_delay : phase delay in range [0(No delay), 1(Full Phase delay)]
    if range_phase_delay < 0.0 or range_phase_delay > 1.0:
        raise ValueError("range_phase_delay must be in range [0, 1]")

    phase_delay = range_phase_delay * freq_rad * 2 * math.pi

    rad_amplitude, rad_median = math.radians(deg_amplitude), math.radians(deg_median)
    rad = rad_amplitude * math.sin(t*freq_rad + phase_delay) + rad_median
    return rad


## Initial Position of Sine Wave Motion
# Calculate initial positions using functionGenerator for all joints
rad_init_pos = []
for i in range(jointnum):
    if i % 3 == 0:  # XX_0 joints (hip)
        rad_init_pos.append(0)  # Hip joints stay at 0
    else:  # XX_1, XX_2 joints (thigh, calf)
        rad_init_pos.append(functionGenerator(0, freq_hz_list[i], deg_sineamplitude[i], deg_sinemedian[i], phase_delay_list[i]))





## Send motion to robot
def motionArr(mCmdArr, joint, pos, vel, kp, kd, tau):
    """
    joint | pos : q | vel : dq | kp : Kp | kd : Kd | tau : ff torque
    each parameter is list for each joint [joint1, joint2, joint3] 
        -> in this code : [FR_0, FR_1, FR_2]

    Returns:
        mCmdArr: array of motorCmd to be set in lowCmd
    """
    for i in range(len(joint)):
        mCmdArr.setMotorCmd(joint[i],  motorCmd(mode=MotorModeLow.Servo, q=pos[i], dq = vel[i], Kp = kp[i], Kd = kd[i], tau = tau[i]))
    return mCmdArr




Running lib version: 0.2
control Freq : 500.0 Hz
num of controlled joints : 12
Joint Frequency Config (Hz): [0.0, 1, 1, 0.0, 1, 1, 0.0, 1, 1, 0.0, 1, 1]
Joint Phase Delay Config: [0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.5, 0.5, 0.0, 0.75, 0.75]
Sine Amplitude (deg) : [0.0, 27.0, 20.0, 0.0, 27.0, 20.0, 0.0, 27.0, 20.0, 0.0, 27.0, 20.0]
Sine Median (deg)    : [0.0, 21.0, -68.0, 0.0, 21.0, -68.0, 0.0, 21.0, -68.0, 0.0, 21.0, -68.0]


In [50]:
## 디버깅용 함수
def print_joint_angles(conn,lstate):
    while True:
        print(
            [round(math.degrees(q),2) for q in get_joint_angles_rad(conn,lstate)]
        )
        time.sleep(0.002) # Sleep 2ms
print_joint_angles(conn,lstate)

[-18.09, 67.89, -158.12, 21.4, 66.65, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.97, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.97, 18.02, 69.08, -157.59, -15.2, 68.11, -164.59]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.59]
[-18.09, 67.89, -158.12, 21.4, 66.65, -158.97, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.89, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
[-18.09, 67.9, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.59]
[-18.09, 67.9, -158.12, 21.4, 66.64, -158.98, 18.02, 69.08, -157.59, -15.2, 68.11, -164.59]


KeyboardInterrupt: 

In [26]:
qInit
qInitdeg = [math.degrees(q) for q in qInit]

[0, 1.1991719007492065, -2.762320041656494, 0, 1.1788253784179688, -2.78460431098938, 0, 1.224786639213562, -2.880362033843994, 0, 1.2346570491790771, -2.7804057598114014]


In [55]:
## 자주쓰는 함수들 정의
# 12개 관절의 각도(q)를 radian 단위의 리스트로 리턴하는 함수
def get_joint_angles_rad(conn,lstate):
    """
    conn에서 최신 패킷을 받아 12개 관절의 각도(q, radian) 리스트를 반환

    Returns:
        q_list (list): 12개 관절의 각도 리스트 (radian, joint 순서는 global 'joint' 리스트와 동일)
                       오류시 None 반환
    """
    joint_isaac = [
    # IsaacLab 에서 사용되는 Joint 순서
    'FL_0', 'FL_1', 'FL_2', # FL_hip, FL_thigh, FL_calf
    'FR_0', 'FR_1', 'FR_2', # FR_hip, FR_thigh, FR_calf
    'RL_0', 'RL_1', 'RL_2', # RL_hip, RL_thigh, RL_calf
    'RR_0', 'RR_1', 'RR_2'  # RR_hip, RR_thigh, RR_calf
    ]
    joint = joint_isaac
    data = conn.getData()
    try:
        lstate.parseData(data[-1])# 최신 패킷만 처리
        # 관절 이름이 'joint' 리스트에 있다고 가정
        q_list = [lstate.motorState[d[jname]].q for jname in joint]
        return q_list

    except Exception as e:
        print(f"관절 각도 읽기 오류: {e}")
        # 만약 'list index out of range' 에러면 너무 빨리 요청한거라 잠시 대기 후 재시도
        if 'list index out of range' in str(e) and len(data) < 1:
            time.sleep(0.002)
            return get_joint_angles_rad(conn, lstate)
        else:
            return None

def doMotion(qList,mCmdArr = mCmdArr,joint = joint, kp = kp, kd = kd, tau = tau):
    ## qList 에 조인트 절대값 넣으면 움직임
    # Hardcode Vars for Test
    joint = ['FR_0', 'FR_1', 'FR_2','FL_0', 'FL_1', 'FL_2','RR_0', 'RR_1', 'RR_2','RL_0', 'RL_1', 'RL_2']
    joint_isaac = [
    # IsaacLab 에서 사용되는 Joint 순서
    'FL_0', 'FL_1', 'FL_2', # FL_hip, FL_thigh, FL_calf
    'FR_0', 'FR_1', 'FR_2', # FR_hip, FR_thigh, FR_calf
    'RL_0', 'RL_1', 'RL_2', # RL_hip, RL_thigh, RL_calf
    'RR_0', 'RR_1', 'RR_2'  # RR_hip, RR_thigh, RR_calf
    ]
    joint = joint_isaac

    jointnum = len(joint)
    kp = [20] * jointnum
    kd = [4] * jointnum
    tau = [2] * jointnum

    lcmd.motorCmd = motionArr(mCmdArr, joint, qDes, [0]*jointnum, kp, kd, tau)
    # build command bytes
    cmd_bytes = lcmd.buildCmd(debug=False)
    # send command bytes to robot
    conn.send(cmd_bytes)

## Linear interpolation between two joint positions
# Input : initPos(rate=0) ~~~~~~ targetPos(rate=1)
# Output : interpolated_position =(p)
def jointLinearInterpolationList(initPos, targetPos, rate):
    # list to np.array for vectorized operation
    np_initPos = np.array(initPos)
    np_targetPos = np.array(targetPos)
    # Clamp rate between 0 and 1
    rate = np.fmin(np.fmax(rate, 0.0), 1.0)
    # Do Interpolation
    np_p = np_initPos*(1-rate) + np_targetPos*rate
    p = np_p.tolist()
    return p

def set_hip_zero(q):
    # 3의 배수 인덱스(즉, 각 관절 그룹의 첫 번째)에 대해 0으로 설정하고 나머지는 기존 값을 유지
    return [0 if i % 3 == 0 else q[i] for i in range(len(q))]

In [ ]:
## 1. get curr angle
# Curr Angle
qInit = get_joint_angles_rad(conn,lstate)
# Set Hip Zero
qInit_cute = set_hip_zero(qInit)
print("Curr Angle : ",[round(math.degrees(q),2) for q in qInit])
print("1stage Target Angle : ",[round(math.degrees(q),2) for q in qInit_cute])


Curr Angle :  [-18.09, 67.89, -158.13, 21.4, 66.64, -158.97, 18.02, 69.08, -157.59, -15.2, 68.11, -164.6]
1stage Target Angle :  [0.0, 67.89, -158.13, 0.0, 66.64, -158.97, 0.0, 69.08, -157.59, 0.0, 68.11, -164.6]


KeyboardInterrupt: 

In [ ]:
joint_isaac = [
    # IsaacLab 에서 사용되는 Joint 순서
    'FL_0', 'FL_1', 'FL_2', # FL_hip, FL_thigh, FL_calf
    'FR_0', 'FR_1', 'FR_2', # FR_hip, FR_thigh, FR_calf
    'RL_0', 'RL_1', 'RL_2', # RL_hip, RL_thigh, RL_calf
    'RR_0', 'RR_1', 'RR_2'  # RR_hip, RR_thigh, RR_calf
]

[0.32197123765945435,
 1.2545796632766724,
 -2.783191204071045,
 -0.24343134462833405,
 1.2248471975326538,
 -2.778266191482544,
 0.4106239080429077,
 1.2949093580245972,
 -2.732769012451172,
 -0.2761915922164917,
 1.2509464025497437,
 -2.8798370361328125]

In [117]:
## Do action 1stage (move Joint to Target Angle)
hz = 50 #[Hz]
duration = 2 #[Sec]

step = int(duration*hz)
for i in range(step):
    qDes = jointLinearInterpolationList(qInit,qInit_cute,i/step)
    ## Debug Print
    # print("target : ",[round(math.degrees(q),2) for q in qDes])
    doMotion(qDes)
    time.sleep(1/hz)
while True:
    doMotion(qInit_cute)
    time.sleep(1/hz)


KeyboardInterrupt: 

In [ ]:
## 1. move leg to standup pos - effect to joint 1,2
def set_joint12_standup(q):
    # 1번쨰 관절 일괄 10도감소
    # 2번쨰 관절 일괄 60도증가
    q = [q[i] - math.radians(20)  if i % 3 == 1 else q[i] for i in range(len(q))] 
    q = [q[i] + math.radians(75)  if i % 3 == 2 else q[i] for i in range(len(q))] 
    return q
qStandup = set_joint12_standup(qInit_cute)

print("1stage Target Angle : ",[round(math.degrees(q),2) for q in qInit_cute])
print("2stage Target Angle : ",[round(math.degrees(q),2) for q in qStandup])
qStandup = default_joint_pos # Isaaclab 에서 일어나는 Pose 에 할당하기


1stage Target Angle :  [0.0, 67.89, -158.13, 0.0, 66.64, -158.97, 0.0, 69.08, -157.59, 0.0, 68.11, -164.6]
2stage Target Angle :  [0.0, 47.89, -83.13, 0.0, 46.64, -83.97, 0.0, 49.08, -82.59, 0.0, 48.11, -89.6]


In [69]:
## Do action 1+2stage (move Joint to Target Angle)
hz = 50 #[Hz]
duration_st1 = 0.5 #[Sec]
duration_st2 = 0.5 #[Sec]

step_st1 = int(duration_st1*hz)
print("step_st1 : ",step_st1)
step_st2 = int(duration_st2*hz)
print("step_st2 : ",step_st2)
torque = 25
tau = [torque]*12


for i in range(step_st1):
    qDes = jointLinearInterpolationList(qInit,qInit_cute,i/step_st1)
    ## Debug Print
    # print("target : ",[round(math.degrees(q),2) for q in qDes])
    doMotion(qDes,tau = tau)
    time.sleep(1/hz)
print("step_st1 done")

for i in range(step_st2):
    qDes = jointLinearInterpolationList(qInit_cute,qStandup,i/step_st2)
    ## Debug Print
    # print("target : ",[round(math.degrees(q),2) for q in qDes])
    doMotion(qDes,tau = tau)
    time.sleep(1/hz)
print("step_st2 done")

# for i in range(step_st1):
#     qDes = jointLinearInterpolationList(qStandup,qInit,i/step_st1)
#     ## Debug Print
#     # print("target : ",[round(math.degrees(q),2) for q in qDes])
#     doMotion(qDes,tau = [0.5]*12)
#     time.sleep(1/hz)
# print("Standdown done")


while True:
    doMotion(qStandup,tau = tau)
    time.sleep(1/hz)

step_st1 :  25
step_st2 :  25
step_st1 done
step_st2 done


KeyboardInterrupt: 

In [61]:
default_joint_pos = [
    0.1,   # FL_hip
    0.8,   # FL_thigh
    -1.5,  # FL_calf
    -0.1,  # FR_hip
    0.8,   # FR_thigh
    -1.5,  # FR_calf
    0.1,   # RL_hip
    1.0,   # RL_thigh
    -1.5,  # RL_calf
    -0.1,  # RR_hip
    1.0,   # RR_thigh
    -1.5   # RR_calf
]


In [65]:
import time
time.sleep(8)
timer = time.time()
while timer - time.time() < 5:
    doMotion(qStandup,tau = tau)
    time.sleep(1/hz)
timer = time.time()

while timer - time.time() < 5:
    doMotion(default_joint_pos,tau = tau)
    time.sleep(1/hz)

timer = time.time()
while timer - time.time() < 5:
    doMotion(qStandup,tau = tau)
    time.sleep(1/hz)


KeyboardInterrupt: 

In [6]:
jointd = list(d.keys())
joint = ['FR_0', 'FR_1', 'FR_2', 'FL_0', 'FL_1', 'FL_2', 'RL_0', 'RL_1', 'RL_2', 'RR_0', 'RR_1', 'RR_2']


In [46]:
while True:
    print(get_RPY(conn,lstate))




[-0.006501940079033375, -0.0018402624409645796, -0.9602348208427429]
imu 읽기 오류: list index out of range
[-0.006523372605443001, -0.0018462621374055743, -0.9602351188659668]
imu 읽기 오류: list index out of range
[-0.006512652616947889, -0.001836499897763133, -0.9602317810058594]
imu 읽기 오류: list index out of range
[-0.0065187616273760796, -0.0018374978099018335, -0.9602251648902893]
imu 읽기 오류: list index out of range
[-0.0065213898196816444, -0.0018365306314080954, -0.9602258801460266]
imu 읽기 오류: list index out of range
[-0.006530177779495716, -0.0018345079151913524, -0.9602351188659668]
imu 읽기 오류: list index out of range
[-0.006518435198813677, -0.0018236959585919976, -0.9602287411689758]
imu 읽기 오류: list index out of range
[-0.006526578683406115, -0.0018204536754637957, -0.9602293968200684]
imu 읽기 오류: list index out of range
[-0.00653066486120224, -0.001825573155656457, -0.9602237343788147]
imu 읽기 오류: list index out of range
[-0.006525238510221243, -0.0018346896395087242, -0.96021372079849

KeyboardInterrupt: 

[-0.006603737827390432, -0.00157430290710181, -0.9596647024154663]

In [ ]:
# 12개 관절의 각도(q)를 radian 단위의 리스트로 리턴하는 함수
def get_RPY(conn,lstate,deg = False):
    """
    conn에서 최신 패킷을 받아 로봇의 imu RPY 각도를 리턴
    
        rpy (list): 로봇의 imu RPY 각도 리스트 (roll, pitch, yaw)
    """
    data = conn.getData()
    try:
        lstate.parseData(data[-1])# 최신 패킷만 처리
        # data 에서 rpy 파싱
        rpy = lstate.imu.rpy
        ## Debug Print
        # print("RPY : ",np.round(np.rad2deg(rpy),2))

        if deg: # deg 플래그가 True 이면 라디안을 도로 변환
            return np.rad2deg(rpy)
        else: # deg 플래그가 False 이면 라디안 그대로 리턴
            return rpy

    except Exception as e:
        print(f"imu 읽기 오류: {e}")
        # 만약 'list index out of range' 에러면 너무 빨리 요청한거라 잠시 대기 후 재시도
        if 'list index out of range' in str(e) and len(data) < 1:
            time.sleep(0.002)
            return get_RPY(conn, lstate)
        else:
            return None

In [ ]:
while True:
    time.sleep(dt)
    motiontime += 1
    
    # Get Data from robot at every cycle
    data = conn.getData()
    for paket in data:
        lstate.parseData(paket)

    # Do loop
    if( motiontime >= 0):

        ## First Phase 
        # Move to the origin point of a sine movement with Kp Kd
        if( motiontime >= 0 and motiontime < 10):
            
            # first, get record initial position of robot
            for i in range(len(joint)):
                qInit[i] = lstate.motorState[d[joint[i]]].q

        # second, move to the origin point of a sine movement with Kp Kd
        if( motiontime >= 10 and motiontime < 400):
            rate = (motiontime - 10) / (400-10) # Slowly increase "`rate` 0 -> 1" over "`motiontime` 10 -> 400" for smooth transition to INITIAL POSE
            print(rate)
            for joint_idx in range(len(joint)):
                qDes[joint_idx] = jointLinearInterpolation(qInit[joint_idx], rad_init_pos[joint_idx], rate)


        ## Third Phase
        # Real Robot Motion

        # Do sine wave motion at every control loop time ctrldt
        if( motiontime >= 400 and motiontime % (ctrldt/dt) == 0): # Check if this motiontime is motion Hz (dt =/= ctrldt)
            # get current Time from motiontime, now t = 0 at motiontime = 400
            t = dt*(motiontime - 400) # time variable for sine function control
            
            ## Should be Replaced by Function Generator!!!
            # sin_joint1 = 0.6 * math.sin(t*freq_rad)
            # sin_joint2 = -0.9 * math.sin(t*freq_rad/2)

            # Calculate Joint Positions by Function Generator for all joints
            for i in range(jointnum):
                if i % 3 == 0:  # XX_0 joints (hip) - keep at 0
                    qDes[i] = 0
                else:  # XX_1, XX_2 joints (thigh, calf) - sine wave motion
                    qDes[i] = functionGenerator(t, freq_hz_list[i], deg_sineamplitude[i], deg_sinemedian[i], phase_delay_list[i])
            
            ## Verbose Target Joint Position Print
            print('')
            print('>>> Target Joint Positions (deg)')
            for deschan in qDes:
                print(round(math.degrees(deschan),2))
            print('>>> Current Time : {:.2f} sec'.format(t))
            print('--------------------------------')
            # ====================================
            
        ## Build and send command
        # orgnize motorCmdArray
        """커맨드 배열 생성 함수!!"""
        lcmd.motorCmd = motionArr(mCmdArr, joint, qDes, [0]*jointnum, kp, kd, tau)
        # build command bytes
        cmd_bytes = lcmd.buildCmd(debug=False)
        # send command bytes to robot
        conn.send(cmd_bytes)
        
        # Data logging for tracking performance
        if motiontime >= 400:  # Start logging after initial positioning
            # Initialize start time on first logging
            if start_time is None:
                start_time = time.time()
            
            # Store target positions (in degrees)
            target_positions.append([math.degrees(qDes[i]) for i in range(jointnum)])
            
            # Store actual motor positions (in degrees)
            actual_positions.append([math.degrees(lstate.motorState[d[joint[i]]].q) for i in range(jointnum)])
            
            # Store both timestamp types
            timestamps.append(motiontime * dt)  # dt-based time
            real_times.append(time.time() - start_time)  # real time elapsed
        
        # -- End of Robot Control -- #